# FieldStomav1 Stomata Area Calculation

## Instructions
1. Download the weights and test images from the following Google Drive link:
   [Google Drive]
   (https://drive.google.com/drive/folders/1jYga1tsGj7DyVwska3clroRCkT_c0MZd?usp=drive_link)

2. Place the `model_final.pth` and `config8.yaml` files in the `models` directory.

3. Install Detectron2:
   ```sh
   python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


In [ ]:
import os
import cv2
import pandas as pd
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
import numpy as np

def load_configuration(config_yaml_path, model_weights_path):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
    cfg.merge_from_file(config_yaml_path)
    cfg.MODEL.WEIGHTS = model_weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    return cfg

def draw_scale_bar(image, bar_length_mm, pixel_density, bar_height, bar_color, text_color, location):
    bar_length_px = int(bar_length_mm * pixel_density)  # Convert mm to pixels
    h, w = image.shape[:2]
    if location == 'bottom_right':
        start_point = (w - bar_length_px - 20, h - bar_height - 20)
    elif location == 'bottom_left':
        start_point = (20, h - bar_height - 20)
    elif location == 'top_right':
        start_point = (w - bar_length_px - 20, 20)
    else:  
        start_point = (20, 20)

    end_point = (start_point[0] + bar_length_px, start_point[1] + bar_height)
    cv2.rectangle(image, start_point, end_point, bar_color, -1)
    cv2.putText(image, f'{bar_length_mm} mm', (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 2)

def detect_and_save_images(cfg, input_folder, output_folder, csv_file, pixel_density):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    predictor = DefaultPredictor(cfg)
    results = []

    
    MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).set(thing_classes=["stomata"])

    for image_name in os.listdir(input_folder):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, image_name)
            image = cv2.imread(image_path)

            outputs = predictor(image)
            instances = outputs["instances"].to("cpu")
            masks = instances.pred_masks if instances.has("pred_masks") else None
            scores = instances.scores if instances.has("scores") else None

            if masks is not None:
                for i, (mask, score) in enumerate(zip(masks, scores)):
                    stomata_area_pixels = np.sum(mask.numpy())
                    stomata_area_um2 = (stomata_area_pixels / (pixel_density ** 2)) * (1000 ** 2)
                    result = {
                        "image_name": image_name,
                        "stomata_number": i + 1,
                        "confidence_score": score.item(),
                        "stomata_area_um2": stomata_area_um2  # Area in square micrometers
                    }
                    results.append(result)

           
            v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2, instance_mode=ColorMode.SEGMENTATION)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            image_with_annotations = out.get_image()[:, :, ::-1]

           
            image_with_annotations = cv2.cvtColor(image_with_annotations, cv2.COLOR_RGB2BGR)

            output_image_path = os.path.join(output_folder, image_name)

           
            bar_length_mm = 1  
            bar_height = 10
            bar_color = (255, 255, 255)
            text_color = (0, 0, 0)
            draw_scale_bar(image_with_annotations, bar_length_mm, pixel_density, bar_height, bar_color, text_color, 'bottom_right')

            cv2.imwrite(output_image_path, image_with_annotations)

    df = pd.DataFrame(results)
    df.to_csv(csv_file, index=False)
    print(f"CSV file saved to {csv_file}")

def main():
    input_folder = "../images"
    output_folder = "../images_detected"
    config_yaml_path = "../models/config.yaml"
    model_weights_path = "../models/model_weights.pth"
    csv_file = "../detection_results.csv"
    pixel_density = 652

    cfg = load_configuration(config_yaml_path, model_weights_path)
    detect_and_save_images(cfg, input_folder, output_folder, csv_file, pixel_density)

if __name__ == "__main__":
    main()